In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import pyarrow
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Replace 'your_folder' with the actual folder path where your Parquet file is located
icc_path = '/content/drive/MyDrive/Precedent/data/'

In [ ]:
# Replace 'your_file.parquet' with the actual Parquet file name
train_file = 'icc_train.parquet'
train_file_path = icc_path + train_file

# Read the Parquet file into a Pandas DataFrame
data = pd.read_parquet(train_file_path, engine='pyarrow')

In [ ]:
data.insert(0, 'id', range(len(data)))

In [ ]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and punctuation
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
    # Rejoin tokens into a clean text
    return ' '.join(tokens)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
judgments = data[:5]

In [ ]:
judgments['cleaned_text'] = judgments['text'].apply(preprocess_text)

<ipython-input-99-5eb49acb5d64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judgments['cleaned_text'] = judgments['text'].apply(preprocess_text)


In [ ]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the TF-IDF vectorizer on the cleaned text data
tfidf_matrix = tfidf_vectorizer.fit_transform(judgments['cleaned_text'])

# Process the current case's text
current_case_text = judgments['text'][0]
current_case_text = preprocess_text(current_case_text)

# Transform the current case text into TF-IDF representation
current_case_vector = tfidf_vectorizer.transform([current_case_text])

# Calculate cosine similarity between current case and previous judgments
cosine_similarities = linear_kernel(current_case_vector, tfidf_matrix)

# Get the top N most similar judgments
N = 2  # You can adjust this based on your preference
similar_judgment_indices = cosine_similarities[0].argsort()[:-N-1:-1]

In [ ]:
# Retrieve and print the most similar judgments
similar_judgments = judgments.iloc[similar_judgment_indices]
for index, judgment in similar_judgments.iterrows():
    print(judgment["id"], judgment['text'])

0 CIVIL APPELLATE JURISDICTION Civil Appeal No. 3838 of 1988. From the Judgment and Order dated 22.6.1987 of the Tamil Nadu High Court in W.A. No. 555 of 1984. T.S. Tulsi, Additional Solicitor General, B.K. Prasad, K. Srivastava, P. Parmeshwaran for the Appellants. T.M. Sampath and K.V. Sreekumar for the Respondents. The Judgment of the Court was delivered by VERMA, J. This matter brings to the fore once again the ineptitude with which litigation is companyducted quite often on behalf of the Government of India and State Governments even when important issues having lasting and wide repercussions are involved. The point in this case relates to the validity of a policy of the railway administration and is likely to affect the staff pattern in several units. Inspite of this fact, to support validity of the impugned policy the required materials were number produced in the High Court and to overcome the adverse decision several opportunities given by us to produce the entire relevant reco

In [ ]:
judgments['text'][0]

'CIVIL APPELLATE JURISDICTION Civil Appeal No. 3838 of 1988. From the Judgment and Order dated 22.6.1987 of the Tamil Nadu High Court in W.A. No. 555 of 1984. T.S. Tulsi, Additional Solicitor General, B.K. Prasad, K. Srivastava, P. Parmeshwaran for the Appellants. T.M. Sampath and K.V. Sreekumar for the Respondents. The Judgment of the Court was delivered by VERMA, J. This matter brings to the fore once again the ineptitude with which litigation is companyducted quite often on behalf of the Government of India and State Governments even when important issues having lasting and wide repercussions are involved. The point in this case relates to the validity of a policy of the railway administration and is likely to affect the staff pattern in several units. Inspite of this fact, to support validity of the impugned policy the required materials were number produced in the High Court and to overcome the adverse decision several opportunities given by us to produce the entire relevant recor